In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier as GBC 
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import *
import random

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df = pd.read_csv('gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/Dataset/rgb_img_dataset_test.csv')

In [4]:
df = df.drop('filename', axis=1)

In [5]:
df

,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,g_range_3,b_range_3,proportion_3,r_range_4,g_range_4,b_range_4,proportion_4,r_range_5,g_range_5,b_range_5,proportion_5,r_range_6,g_range_6,b_range_6,proportion_6,r_range_7,g_range_7,b_range_7,proportion_7,label
0,16-31,16-31,16-31,0.303225,0-15,0-15,0-15,0.235109,16-31,0-15,0-15,0.050533,16-31,16-31,0-15,0.040913,32-47,16-31,16-31,0.026910,176-191,128-143,96-111,0.020603,192-207,144-159,96-111,0.018866,0
1,16-31,16-31,16-31,0.176878,240-255,240-255,240-255,0.091829,224-239,224-239,224-239,0.076498,0-15,0-15,0-15,0.073471,240-255,224-239,224-239,0.039134,240-255,240-255,224-239,0.035281,208-223,208-223,208-223,0.028382,0
2,224-239,224-239,224-239,0.092203,208-223,208-223,208-223,0.082258,240-255,224-239,224-239,0.065342,192-207,192-207,192-207,0.046121,80-95,80-95,80-95,0.039306,176-191,176-191,176-191,0.039293,64-79,64-79,64-79,0.038894,0
3,32-47,32-47,48-63,0.063760,0-15,32-47,112-127,0.059953,0-15,48-63,128-143,0.055638,0-15,48-63,144-159,0.042434,0-15,32-47,96-111,0.036150,240-255,240-255,240-255,0.033036,48-63,48-63,64-79,0.032069,0
4,0-15,0-15,0-15,0.127613,160-175,176-191,192-207,0.112392,144-159,176-191,192-207,0.067298,176-191,192-207,208-223,0.067047,144-159,160-175,192-207,0.059518,16-31,16-31,0-15,0.050183,16-31,0-15,0-15,0.035422,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,16-31,0-15,0-15,0.382456,16-31,16-31,0-15,0.152202,96-111,96-111,112-127,0.019563,32-47,16-31,0-15,0.019206,80-95,80-95,96-111,0.018939,112-127,112-127,128-143,0.018107,64-79,64-79,80-95,0.015809,0
606,32-47,32-47,32-47,0.196324,192-207,192-207,192-207,0.066635,176-191,176-191,176-191,0.037899,208-223,208-223,208-223,0.029360,16-31,16-31,16-31,0.028479,192-207,192-207,176-191,0.026092,48-63,48-63,48-63,0.024403,0
607,128-143,112-127,64-79,0.030152,144-159,112-127,48-63,0.028196,144-159,112-127,64-79,0.026224,112-127,96-111,48-63,0.024859,128-143,96-111,48-63,0.023839,160-175,128-143,64-79,0.023400,128-143,96-111,32-47,0.022496,0
608,0-15,32-47,32-47,0.134387,16-31,32-47,32-47,0.129191,16-31,32-47,16-31,0.048654,64-79,112-127,144-159,0.043070,32-47,48-63,48-63,0.039828,48-63,112-127,128-143,0.032310,48-63,48-63,48-63,0.024963,0


In [6]:
def convert_range_to_order(val):
    result = int(val.split('-')[0])/16
    return int(result)

In [7]:
for col_name in df.columns:
    if('_range_' in col_name):
        df[col_name] = df[col_name].apply(lambda x: convert_range_to_order(x))

In [8]:
df

,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,g_range_3,b_range_3,proportion_3,r_range_4,g_range_4,b_range_4,proportion_4,r_range_5,g_range_5,b_range_5,proportion_5,r_range_6,g_range_6,b_range_6,proportion_6,r_range_7,g_range_7,b_range_7,proportion_7,label
0,1,1,1,0.303225,0,0,0,0.235109,1,0,0,0.050533,1,1,0,0.040913,2,1,1,0.026910,11,8,6,0.020603,12,9,6,0.018866,0
1,1,1,1,0.176878,15,15,15,0.091829,14,14,14,0.076498,0,0,0,0.073471,15,14,14,0.039134,15,15,14,0.035281,13,13,13,0.028382,0
2,14,14,14,0.092203,13,13,13,0.082258,15,14,14,0.065342,12,12,12,0.046121,5,5,5,0.039306,11,11,11,0.039293,4,4,4,0.038894,0
3,2,2,3,0.063760,0,2,7,0.059953,0,3,8,0.055638,0,3,9,0.042434,0,2,6,0.036150,15,15,15,0.033036,3,3,4,0.032069,0
4,0,0,0,0.127613,10,11,12,0.112392,9,11,12,0.067298,11,12,13,0.067047,9,10,12,0.059518,1,1,0,0.050183,1,0,0,0.035422,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,1,0,0,0.382456,1,1,0,0.152202,6,6,7,0.019563,2,1,0,0.019206,5,5,6,0.018939,7,7,8,0.018107,4,4,5,0.015809,0
606,2,2,2,0.196324,12,12,12,0.066635,11,11,11,0.037899,13,13,13,0.029360,1,1,1,0.028479,12,12,11,0.026092,3,3,3,0.024403,0
607,8,7,4,0.030152,9,7,3,0.028196,9,7,4,0.026224,7,6,3,0.024859,8,6,3,0.023839,10,8,4,0.023400,8,6,2,0.022496,0
608,0,2,2,0.134387,1,2,2,0.129191,1,2,1,0.048654,4,7,9,0.043070,2,3,3,0.039828,3,7,8,0.032310,3,3,3,0.024963,0


In [9]:
df['label'].value_counts()

0    456
1    154
Name: label, dtype: int64

In [10]:
def model_training(df, training_size, param): 
    # training_size is ratio of training data for train-test split (float)
    # param is set of parameter for model training (dict)
    
    X = df.drop('label', axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = tts(X, y, train_size=training_size, random_state=42)
    
    sm = SMOTE(sampling_strategy = 1, random_state=42)
    X_res, y_res = sm.fit_resample(X_train, y_train.ravel())
    
    clf = GBC(max_depth=param['max_depth'], 
              criterion=param['criterion'],
              min_impurity_decrease=param['min_impurity_decrease'], random_state=42).fit(X_res, y_res)
    # clf = GBC(set_of_parameter)
    y_pred = clf.predict(X_test)
    
    print('Confusion Matrix')
    print(confusion_matrix(y_test, y_pred))
    print('Accuracy: {:.4f}'.format(accuracy_score(y_test, y_pred)))
    print('Precision: {:.4f}'.format(precision_score(y_test, y_pred)))
    print('Recall: {:.4f}'.format(recall_score(y_test, y_pred)))
    
    return clf

In [15]:
def random_parameter(param_name):
    
    if(param_name=='max_depth'):
        return random.randint(1, 10)
    
    if(param_name=='criterion'):
        tmp = random.randint(1, 2)
        if(tmp==1):
            return 'friedman_mse'
        elif(tmp==2):
            return 'mae'
            
    if(param_name=='min_impurity_decrease'):
        return random.random()/10

In [16]:
param = {'max_depth': random_parameter('max_depth'),
         'criterion': random_parameter('criterion'),
         'min_impurity_decrease': random_parameter('min_impurity_decrease')}

In [17]:
model = model_training(df, 0.75, param)

Confusion Matrix
[[66 51]
 [ 7 29]]
Accuracy: 0.6209
Precision: 0.3625
Recall: 0.8056


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:475: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: C

In [18]:
for i in range(5):
    param = {'max_depth': random_parameter('max_depth'),
             'criterion': random_parameter('criterion'),
             'min_impurity_decrease': random_parameter('min_impurity_decrease')}
    model = model_training(df, 0.75, param)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:475: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,


Confusion Matrix
[[117   0]
 [ 36   0]]
Accuracy: 0.7647
Precision: 0.0000
Recall: 0.0000


/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: 

Confusion Matrix
[[91 26]
 [15 21]]
Accuracy: 0.7320
Precision: 0.4468
Recall: 0.5833
Confusion Matrix
[[91 26]
 [15 21]]
Accuracy: 0.7320
Precision: 0.4468
Recall: 0.5833
Confusion Matrix
[[95 22]
 [11 25]]
Accuracy: 0.7843
Precision: 0.5319
Recall: 0.6944
Confusion Matrix
[[66 51]
 [ 7 29]]
Accuracy: 0.6209
Precision: 0.3625
Recall: 0.8056


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:475: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: C